In [1]:
using JuMP
using NLopt
using LinearAlgebra

using HDF5

using QuantumOptics
basis = NLevelBasis(2)

include("LiPoSID.jl")

using Statistics

using Dates

In [2]:
function det2x2(m)
    real(m[1,1]*m[2,2] - m[1,2]*m[2,1])
end

det2x2 (generic function with 1 method)

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

In [3]:
function infidelity_norm(ρ, σ)
    abs(1 - real(tr(ρ * σ)) - 2*sqrt(abs(det(ρ)*det(σ))))#^2
end

function fidelity(ρ, σ)
    real(tr(ρ * σ)) + 2*sqrt(abs(det(ρ)*det(σ)))#^2
end

function frobenius_norm2(m)
    return abs(tr(m * m'))
end

frobenius_norm2 (generic function with 1 method)

In [31]:
# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function DcKossak(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end


# Function to calculate Dc
function DcLindblad(ρ, t, H, J)
    U = (H * ρ - ρ * H) / im
    D = sum([ ( Jⱼ * ρ * Jⱼ' - (Jⱼ' * Jⱼ  * ρ + ρ * Jⱼ' * Jⱼ)/2 ) for Jⱼ in J ])
    return U + D
end


# Function to calculate Dc
#function DcLindblad(ρ, t, H, J1)
#    U = (H * ρ - ρ * H) / im
#    D = ( J1 * ρ * J1' - (J1' * J1  * ρ + ρ * J1' * J1)/2 ) 
#    return U + D
#end


DcLindblad (generic function with 1 method)

### Kossakowski objective for two-level system

A linear operator $L: M(2) \rightarrow M(2)$ is the generator of a dynamical semigroup $t \rightarrow \Lambda_t$ of $M(2)$ iff it can be written in the form:

$L = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, \quad \rho \in M(2)$

where

(i) $H = \sum_1^3 h_i f_i, \quad h_i \in \R$

(ii) $f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k$ which means:

$ \operatorname{tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{tr}(f_i) = 0$

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} \epsilon & \alpha + i \beta \\ \alpha - i \beta  & -\epsilon
   \end{pmatrix}/2
$

here we consider the following set of jump operators ot the particular ansatz form:

$
J_1 = \begin{pmatrix} 0 & r \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$  - for relaxation process, 

and three following operators for depolarization process:

$
J_2 = \begin{pmatrix} 0 & p \\ p & 0 
   \end{pmatrix} = \sqrt{\gamma_4/3} 
   \begin{pmatrix} 0 & 1\\ 1 & 0 
   \end{pmatrix}
$

$
J_3 = \begin{pmatrix} 0 & -i p \\ i p & 0 
   \end{pmatrix} = \sqrt{\gamma_4/3}
   \begin{pmatrix} 0 & -i \\ i & 0 
   \end{pmatrix}
$

$
J_4 = \begin{pmatrix} p & 0 \\ 0 & -p
   \end{pmatrix} = \sqrt{\gamma_4/3}
   \begin{pmatrix} 1 & 0 \\ 0 & -1 
   \end{pmatrix}
$

In [5]:
# Define functions to construct H and C from parameters
function construct_H(ϵ, h_Re, h_Im)
    return [
             ϵ                     h_Re + im * h_Im
             h_Re - im * h_Im     -ϵ
           ] / 2
end

function construct_C(γ, a)
    return [
        -γ[1] + γ[2] + γ[3]  -im * a[3]            im * a[2]
        im * a[3]            γ[1] - γ[2] + γ[3]   -im * a[1]
        -im * a[2]           im * a[1]             γ[1] + γ[2] - γ[3]
    ]
end

function construct_J1(r)
    return [
         0   r
         0   0    
    ]
end

function construct_J2(p)
    return [
         0   p
         p   0    
    ]
end

function construct_J3(p)
    return [
         0     -im*p
         im*p   0    
    ]
end

function construct_J4(p)
    return [
         p    0
         0   -p   
    ]
end

construct_J4 (generic function with 1 method)

In [6]:
# Objective function

function kossak_obj(ϵ, h_Re, h_Im, γ, a, ρ, t) #Simpson 
    H = construct_H(ϵ, h_Re, h_Im)
    C = construct_C(γ, a)
    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + 
        (t[i] - t[i - 1]) * 
        (DcKossak(ρ[i], t[i], H, C) +
         4 * DcKossak(ρ[i - 1], t[i - 1], H, C) 
         + DcKossak(ρ[i - 2], t[i - 2], H, C)) / 3
        #obj += infidelity_norm(ρ1,ρ2)
        #obj += fidelity(ρ1,ρ2)
        obj += frobenius_norm2(ρ1-ρ2)
    end
    return obj
end

# Define the objective function wrapper
function kossak_objectiveGEXY(ϵ, h_Re, h_Im, γ1, γ2, γ3, a1, a2, a3)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    γ = [γ1, γ2, γ3]
    a = [a1, a2, a3]
    objGEXY = kossak_obj(ϵ, h_Re, h_Im, γ, a, ρᵍ, tᵍ) + 
              kossak_obj(ϵ, h_Re, h_Im, γ, a, ρᵉ, tᵉ) + 
              kossak_obj(ϵ, h_Re, h_Im, γ, a, ρˣ, tˣ) + 
              kossak_obj(ϵ, h_Re, h_Im, γ, a, ρʸ, tʸ)
    return objGEXY
end


kossak_objectiveGEXY (generic function with 1 method)

In [32]:
function lindblad_obj(ϵ, h_Re, h_Im, r, p, ρ, t) #Simpson 
    H = construct_H(ϵ, h_Re, h_Im)
    J1 = construct_J1(r)
    J2 = construct_J2(p)
    J3 = construct_J3(p)
    J4 = construct_J4(p)

    J = [J1, J2, J3, J4]
    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + 
        (t[i] - t[i - 1]) * 
        (DcLindblad(ρ[i], t[i], H, J1) +
         4 * DcLindblad(ρ[i - 1], t[i - 1], H, J) 
         + DcLindblad(ρ[i - 2], t[i - 2], H, J)) / 3
        #obj += infidelity_norm(ρ1,ρ2)
        #obj += fidelity(ρ1,ρ2)
        obj += frobenius_norm2(ρ1-ρ2)
    end
    return obj
end

lindblad_obj (generic function with 1 method)

In [33]:
# Define the objective function wrapper
function lindblad_objectiveGEXY(ϵ, h_Re, h_Im, r, p)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    objGEXY = lindblad_obj(ϵ, h_Re, h_Im, r, p, ρᵍ, tᵍ) + 
              lindblad_obj(ϵ, h_Re, h_Im, r, p, ρᵉ, tᵉ) + 
              lindblad_obj(ϵ, h_Re, h_Im, r, p, ρˣ, tˣ) + 
              lindblad_obj(ϵ, h_Re, h_Im, r, p, ρʸ, tʸ)
    return objGEXY
end


lindblad_objectiveGEXY (generic function with 1 method)

In [34]:
# Define a function to create and solve the model with given initial guesses and a general upper bound
function solve_lindblad_model(initial_guess, upper_bound) #, γᵢ)

    #upper_bound = 260
    
    model = Model(optimizer_with_attributes(NLopt.Optimizer, "algorithm" => :LD_SLSQP))

    # Register the element-wise power operator
    register(model, :.^, 2, (x, y) -> x.^y, autodiff=true)

    # Register the custom objective function
    register(model, :lindblad_objectiveGEXY, 5, lindblad_objectiveGEXY, autodiff = true)

    # Define lower and upper bounds
    lower_bound = -upper_bound

    # Define variables with general upper and lower bounds and initial guesses
    @variable(model, 0 <= ϵ <= upper_bound, start = initial_guess[1])
    @variable(model, lower_bound <= h_Re <= upper_bound, start = initial_guess[2])
    @variable(model, lower_bound <= h_Im <= upper_bound, start = initial_guess[3])
    @variable(model, lower_bound <= r <= upper_bound, start = initial_guess[4])
    @variable(model, lower_bound <= p <= upper_bound, start = initial_guess[5])

    # Objective function
    @NLobjective(model, Min, lindblad_objectiveGEXY(ϵ, h_Re, h_Im, r, p))

    # Solve the model
    JuMP.optimize!(model)

    # Retrieve and print results
    objective_value = JuMP.objective_value(model)
    ϵ_value = value(ϵ)
    h_Re_value = value(h_Re)
    h_Im_value = value(h_Im)
    r_value = value(r)
    p_value = value(p)


    #println("Initial Guess: ", initial_guess)
    println("Objective Value: ", objective_value)
    println("ϵ: ", ϵ_value)
    println("r: ", r_value)
    println("p: ", p_value)


    return objective_value, ϵ_value, h_Re_value, h_Im_value, r_value, p_value
end

solve_lindblad_model (generic function with 1 method)

In [35]:
# Define the density matrix evolution and time points

evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.25133"
γᶠ = parse(ComplexF64, γᵢ)

tᵍᵉˣʸ , ρᵍᵉˣʸ  = LiPoSID.read_GEXY_timeevolution(evol_data_file_name, γᵢ)

((Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.086666666667, 0.093333333334, 0.1, 0.106666666667, 0.113333333334  …  23.806666666786, 23.813333333452, 23.820000000119, 23.826666666786, 23.833333333452, 23.840000000119, 23.846666666786, 23.853333333453, 23.860000000119, 23.866666666786], Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.086666666667, 0.093333333334, 0.1, 0.106666666667, 0.113333333334  …  23.806666666786, 23.813333333452, 23.820000000119, 23.826666666786, 23.833333333452, 23.840000000119, 23.846666666786, 23.853333333453, 23.860000000119, 23.866666666786], Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.086666666667, 0.093333333334, 0.1, 0.106666666667, 0.113333333334  …  23.806666666786, 23.813333333452, 23.820000000119, 23.826666666786, 23.833333333452, 23.840000000119, 23.846666666786, 23.853333333453, 23.860000000119, 23.866666666786], Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.08666666666

In [38]:
# Define a general upper bound for all variables
upper_bound = 260

#Smart guesses for pure relaxation
smart_guess = zeros(5)
smart_guess[1] = 25.133 # ϵ 
smart_guess[2] = 0.0
smart_guess[3] = 0.0
smart_guess[4] = sqrt(γᶠ)  # γ1
smart_guess[5] = 0.0  # γ2

object_val, ϵ_value, h_Re_value, h_Im_value, r_value, p_value = solve_lindblad_model(smart_guess, upper_bound)

Objective Value: 0.0017010317851748294
ϵ: 25.13648577313258
r: 0.548639831905244
p: 0.0


(0.0017010317851748294, 25.13648577313258, 0.00027882297989523236, -0.00014560326194184892, 0.548639831905244, 0.0)

In [37]:
# Substitute the optimized parameters back into H and C
optimized_H = construct_H(ϵ_value, h_Re_value, h_Im_value)
optimized_J1 = construct_J1(r_value)

2×2 Matrix{Float64}:
 0.0  0.54864
 0.0  0.0

In [40]:
optimized_J2 = construct_J2(p_value)
optimized_J3 = construct_J3(p_value)
optimized_J4 = construct_J4(p_value)

2×2 Matrix{Float64}:
 0.0   0.0
 0.0  -0.0

In [28]:
optimized_H

2×2 Matrix{ComplexF64}:
     12.5703+0.0im         0.000138973-7.38686e-5im
 0.000138973+7.38686e-5im     -12.5703+0.0im

In [44]:
Hˢⁱᵈ = convert.(ComplexF64,optimized_H)
J1ˢⁱᵈ = convert.(ComplexF64,optimized_J1)
J2ˢⁱᵈ = convert.(ComplexF64,optimized_J2)
J3ˢⁱᵈ = convert.(ComplexF64,optimized_J3)
J4ˢⁱᵈ = convert.(ComplexF64,optimized_J4)

#effective_Lindblad = LiPoSID.get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

effective_Lindblad = [J1ˢⁱᵈ, J2ˢⁱᵈ, J3ˢⁱᵈ, J4ˢⁱᵈ]

4-element Vector{Matrix{ComplexF64}}:
 [0.0 + 0.0im 0.548639831905244 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 [0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 [0.0 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 [0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.0 + 0.0im]

In [45]:
ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

In [16]:
#using QuantumOptics
#basis = NLevelBasis(2)
#using Statistics

In [46]:
function test_and_save_D10(Hˢⁱᵈ, Jˢⁱᵈ, γᵢ, output_file_name)

    file_name = "DATA/ALL_GAMMAS_B4_D10.h5"
    γ = ["0.079477", "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]
    #date_and_time_string = string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
    tests_data_file_name = "BENCHMARK_TEST_" * output_file_name * ".h5"

    ρᵍ₀ = [ 1 0.
            0 0 ]    # state to measure initial distance from

    dodeca_10_states = ["D"*string(n) for n=1:10];

    basis_states = ["B"*string(n) for n=1:4];

    train_states = basis_states 
    test_states = dodeca_10_states;

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    h5open(tests_data_file_name, "cw") do fid
        γ_group = create_group(fid, γᵢ) # create coupling group "gamma_" *
        #γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        #γ_group["C"] = convert.(ComplexF64, Cˢⁱᵈ)
    end

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = LiPoSID.read_timeevolution(file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert.(Float64, tₛ)

        ρₒ = DenseOperator(basis,ρₛ[1])
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        #print("effective_Lindblad_ops for Kossakowski")       
        effective_Lindblad_ops = [DenseOperator(basis,j) for j in Jˢⁱᵈ]

        #print("Simulating Kossakowski")
        tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
        ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

        #print("Calculating Fidelity")

        #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
        F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

        h5open(tests_data_file_name, "cw") do fid
            #γ_group = create_group(fid, "gamma_" * γᵢ) # open coupling group
            γ_group = open_group(fid, γᵢ) # open coupling group "gamma_" *
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, F)
        end
        
        FminState = minimum(F)
        FmedianState = median(F)
        FmeanState = mean(F)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)

    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    println()
    println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
    println("Median fidelity for "*γᵢ*": ", F_median_value)

    return(F_min_value, F_median_value)
    
end

test_and_save_D10 (generic function with 1 method)

In [47]:
function test_D10(Hˢⁱᵈ, effective_Lindblad, γᵢ)

    file_name = "DATA/ALL_GAMMAS_B4_D10.h5"

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = LiPoSID.read_timeevolution(file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert.(Float64, tₛ)

        #Simulated LME 
        #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
        #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

        ρₒ = DenseOperator(basis,ρₛ[1])
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        #print("effective_Lindblad_ops for Kossakowski")

        
        effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

        #print("Simulating Kossakowski")

        tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
        ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

        #print("Calculating Fidelity")

        #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
        F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
        
        FminState = minimum(F)
        FmedianState = median(F)
        FmeanState = mean(F)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)

    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    println()
    println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
    println("Median fidelity for "*γᵢ*": ", F_median_value)

    return(F_min_value, F_median_value)
    
end

test_D10 (generic function with 1 method)

In [48]:
test_D10(Hˢⁱᵈ, effective_Lindblad, γᵢ)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9974175358938878
Median fidelity for 0.25133: 0.9992756536246723


(0.9974175358938878, 0.9992756536246723)

In [49]:
γᶠ = real(γᶠ)

0.25133

In [50]:
H_anz = [ 25.14   0+0im
           0.      25.14 ]/2

C_anz = [    γᶠ       im*γᶠ    0.0
            -im*γᶠ      γᶠ      0.0
            0.0        0.0      0.0  ]

ansatz_Lindblad = LiPoSID.get_lindblad_operators(C_anz, fᴼᴺᴮ)

test_D10(Hˢⁱᵈ, ansatz_Lindblad, γᵢ)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9993930860227619
Median fidelity for 0.25133: 0.9999547558360644


(0.9993930860227619, 0.9999547558360644)

In [51]:
J1_anz = [ 0.   sqrt(γᶠ)
           0.    0.     ]
       
test_D10(Hˢⁱᵈ, [J1_anz], γᵢ)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9993930860227618
Median fidelity for 0.25133: 0.9999547558360644


(0.9993930860227618, 0.9999547558360644)

In [53]:
using Dates, Random

In [55]:
date_and_time_string = string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"

tests_dir=""

model_name = "LindbladFrob_Jump_NLOPT_LD_SLSQP"

models_data_file_name = model_name * date_and_time_string 

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in  γ
    println("γ=",γᵢ)

    γᶠ = parse(ComplexF64, γᵢ)

    tᵍᵉˣʸ , ρᵍᵉˣʸ  = LiPoSID.read_GEXY_timeevolution(evol_data_file_name, γᵢ)

    #ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    #tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    # Define a general upper bound for all variables
    upper_bound = 260

    #Smart guesses for pure relaxation
    smart_guess = zeros(5)
    smart_guess[1] = 25.133 # ϵ 
    smart_guess[2] = 0.0
    smart_guess[3] = 0.0
    smart_guess[4] = sqrt(γᶠ)  # r
    smart_guess[5] = 0.0  # p


    elapsed_time = @timed begin

        object_val, ϵ_value, h_Re_value, h_Im_value, r_value, p_value = solve_lindblad_model(smart_guess, upper_bound)
    
    end

    # Substitute the optimized parameters back into H and C
    optimized_H = construct_H(ϵ_value, h_Re_value, h_Im_value)
    optimized_J1 = construct_J1(r_value)
    optimized_J2 = construct_J2(p_value)
    optimized_J3 = construct_J3(p_value)
    optimized_J4 = construct_J4(p_value)

    Hˢⁱᵈ = convert.(ComplexF64,optimized_H)
    J1ˢⁱᵈ = convert.(ComplexF64,optimized_J1)
    J2ˢⁱᵈ = convert.(ComplexF64,optimized_J2)
    J3ˢⁱᵈ = convert.(ComplexF64,optimized_J3)
    J4ˢⁱᵈ = convert.(ComplexF64,optimized_J4)

    effective_Lindblad = [J1ˢⁱᵈ, J2ˢⁱᵈ, J3ˢⁱᵈ, J4ˢⁱᵈ]

    h5open(tests_dir * "MODEL_" * models_data_file_name * ".h5", "cw") do fid
        γ_group = create_group(fid, γᵢ) # create coupling group "gamma_" *
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["J1"] = convert.(ComplexF64, J1ˢⁱᵈ)
        γ_group["J2"] = convert.(ComplexF64, J2ˢⁱᵈ)
        γ_group["J3"] = convert.(ComplexF64, J3ˢⁱᵈ)
        γ_group["J4"] = convert.(ComplexF64, J4ˢⁱᵈ)
        γ_group["runtime"] = elapsed_time.time
    end

    #test_D10(Hˢⁱᵈ, effective_Lindblad, γᵢ)

    test_and_save_D10(Hˢⁱᵈ, effective_Lindblad, γᵢ, models_data_file_name)
    
end

    

γ=0.079477
Objective Value: 0.0013066910221882493
ϵ: 25.122089508069056
r: 0.30848168203649334
p: 0.0
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.079477: 0.9969654423107038
Median fidelity for 0.079477: 0.998112670856418
γ=0.25133
Objective Value: 0.0017010317851748294
ϵ: 25.13648577313258
r: 0.548639831905244
p: 0.0
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9974175358938878
Median fidelity for 0.25133: 0.9992756536246723
γ=0.79477
Objective Value: 0.001967122288200112
ϵ: 25.15314707799559
r: 0.9735337749938767
p: 0.0
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.79477: 0.9973889638475019
Median fidelity for 0.79477: 0.9985306890634686
γ=2.5133
Objective Value: 0.0031599188843957383
ϵ: 25.200229844237267
r: 1.710372191327141
p: 0.0
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 2.5133: 0.9927907618197515
Median fidelity for 2.5133: 0.9960993923832566
γ=7.9477
Objective Value: 0.005008760898741296
ϵ: 24.962122076679176
r: 2.785571163219059